# Load Dataset

In [15]:
import csv

test_images_path = r'/home/daniel/Git/Hot-Topics-in-NLP/notebooks/homeworks/homework_4/test_images_resized'
en_test_data_path = r'/home/daniel/Git/Hot-Topics-in-NLP/notebooks/homeworks/homework_4/test.data.v1.1.gold/en.test.data.v1.1.txt'
en_test_data_gold_path = r'/home/daniel/Git/Hot-Topics-in-NLP/notebooks/homeworks/homework_4/test.data.v1.1.gold/en.test.gold.v1.1.txt'

test_data_path = en_test_data_path
test_data_gold_path = en_test_data_gold_path

# train_images_path = r'/home/daniel/Git/Hot-Topics-in-NLP/notebooks/homeworks/homework_4/train_trial_images/train_v1'

# word, word context, images_list, gold
test_data = []
with open(test_data_path, 'r', encoding='utf-8') as test_data_file, \
    open(test_data_gold_path, 'r', encoding='utf-8') as test_data_gold_file:

    test_data_reader = csv.reader(test_data_file, delimiter='\t')
    test_gold_reader = csv.reader(test_data_gold_file, delimiter='\t')

    for data_line, gold_line in zip(test_data_reader, test_gold_reader):
        test_data.append((data_line[0], data_line[1], data_line[2:], gold_line[0]))

for test_entry in test_data[:10]:
    print(test_entry)

('goal', 'football goal', ['image.4418.jpg', 'image.4416.jpg', 'image.4417.jpg', 'image.4413.jpg', 'image.4412.jpg', 'image.4415.jpg', 'image.4419.jpg', 'image.4414.jpg', 'image.2166.jpg', 'image.1150.jpg'], 'image.2166.jpg')
('mustard', 'mustard seed', ['image.4429.png', 'image.4422.jpg', 'image.4423.jpg', 'image.4424.jpg', 'image.4421.jpg', 'image.4427.jpg', 'image.4426.jpg', 'image.4420.jpg', 'image.4425.jpg', 'image.4428.jpg'], 'image.4429.png')
('seat', 'eating seat', ['image.4435.jpg', 'image.4436.jpg', 'image.1166.jpg', 'image.4430.jpg', 'image.4433.jpg', 'image.4432.jpg', 'image.4438.jpg', 'image.4434.jpg', 'image.4431.jpg', 'image.4437.jpg'], 'image.4432.jpg')
('navigate', 'navigate the web', ['image.4439.jpg', 'image.4440.jpg', 'image.4441.jpg', 'image.4442.jpg', 'image.4444.jpg', 'image.4445.jpg', 'image.1435.jpg', 'image.4446.png', 'image.1434.jpg', 'image.4443.jpg'], 'image.1435.jpg')
('butterball', 'butterball person', ['image.4454.jpg', 'image.4450.jpg', 'image.4455.jpg'

# CLIP Pretrained

In [16]:
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

image.show()

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/daniel/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Given an image, the CLIPVisionModel returns the embedding of the respective text description

In [17]:
from transformers import AutoProcessor, CLIPVisionModel

model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

inputs = processor(images=image, return_tensors="pt")

outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
pooled_output = outputs.pooler_output  # pooled CLS states

# Access the image embeddings
image_embedding = outputs.last_hidden_state 
image_embedding

tensor([[[-0.1391,  0.1016, -0.2362,  ...,  0.1568,  0.2143,  0.2181],
         [ 0.1976,  0.4183, -0.8494,  ...,  0.1030,  1.4616,  0.4321],
         [ 0.1630,  0.2004, -1.0608,  ..., -0.0640,  1.3778,  0.5354],
         ...,
         [ 0.0552, -0.0113, -0.6040,  ...,  0.2935,  1.0303, -0.0316],
         [ 0.1673,  0.0491, -0.3833,  ...,  0.3797,  0.9275, -0.0212],
         [ 0.1937,  0.2567, -0.4325,  ...,  0.2957,  1.2151, -0.0417]]],
       grad_fn=<AddBackward0>)

Given some text, the CLIPTextModel returns an image corresponding to the embedding of the provided text 

In [18]:
from transformers import AutoTokenizer, CLIPTextModel

model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

inputs = tokenizer(["a photo of a cat", "a photo of a dog"], padding=True, return_tensors="pt")

outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
pooled_output = outputs.pooler_output  # pooled (EOS token) states

text_embedding = outputs.last_hidden_state
text_embedding

tensor([[[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 1.9753, -0.5844,  0.3685,  ...,  1.1658,  0.8050, -0.9801],
         [ 1.0580, -0.9600,  1.0018,  ..., -0.5155, -0.1437, -1.9444],
         ...,
         [ 0.3059, -1.5037, -0.4022,  ..., -0.0224,  0.9105, -0.3916],
         [ 1.0118, -0.6701,  1.7742,  ..., -0.1556, -0.0250, -1.5062],
         [-0.5152,  0.1658,  0.8876,  ..., -0.0675, -0.4551, -1.7960]],

        [[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 1.9753, -0.5844,  0.3685,  ...,  1.1658,  0.8050, -0.9801],
         [ 1.0580, -0.9600,  1.0018,  ..., -0.5155, -0.1437, -1.9444],
         ...,
         [ 0.3059, -1.5037, -0.4022,  ..., -0.0224,  0.9105, -0.3916],
         [-0.1433, -0.5163,  1.7099,  ..., -0.0795,  0.3609, -1.2437],
         [ 0.0426,  0.0189,  1.2740,  ..., -0.4217, -0.4393, -1.3016]]],
       grad_fn=<NativeLayerNormBackward0>)

For each row of the dataset we have:
1. the word
2. a minimal context
3. a list of 10 images, among which only one refers to the real word context

We can use the pretrained version of CLIP this way:
1. prompt clip injecting the word and the respective context
2. compute the text embedding for the prompt
3. compute the image embeddings for each of the 10 images
4. measure the cosine similarity between the text embedding and each of the 10 image embeddings,
    ranking them

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def min_cosine(target_vector, vector_list):
    similarities = [cosine_similarity([target_vector], [vector])[0, 0] for vector in vector_list]
    best_index = np.argmin(similarities)
    best_vector = vector_list[best_index]
    return best_vector, similarities[best_index]

def cosine_similarity_key(vector1, vector2):
    return cosine_similarity([vector1], [vector2])[0, 0]

def rank_cosine(target_vector, vector_list):
    sorted_vectors = sorted(vector_list, key=lambda vector: cosine_similarity_key(target_vector, vector[1]), reverse=True)
    return sorted_vectors

"""
# Example usage:
target_vector = np.array([1, 2, 3])  # Replace with your actual target vector
vector_list = [np.array([4, 5, 6]), np.array([7, 8, 9]), np.array([10, 11, 12])]  # Replace with your actual list of vectors

# best_vector, min_similarity = min_cosine(target_vector, vector_list)
print(f"The vector with the smallest cosine similarity is: {best_vector}")
print(f"Cosine similarity value: {min_similarity}")
print(f"Ranked by similarity: {rank_cosine(target_vector, vector_list)}")
"""


'\n# Example usage:\ntarget_vector = np.array([1, 2, 3])  # Replace with your actual target vector\nvector_list = [np.array([4, 5, 6]), np.array([7, 8, 9]), np.array([10, 11, 12])]  # Replace with your actual list of vectors\n\n# best_vector, min_similarity = min_cosine(target_vector, vector_list)\nprint(f"The vector with the smallest cosine similarity is: {best_vector}")\nprint(f"Cosine similarity value: {min_similarity}")\nprint(f"Ranked by similarity: {rank_cosine(target_vector, vector_list)}")\n'

In [35]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import tqdm
import os
import torch

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

predictions = []
with torch.no_grad():
    for word, context, images_list, gold in tqdm.tqdm(test_data):

        prompt = f'This is a picture of {word} in the context of {context}'
    
        all_images = [Image.open(os.path.join(test_images_path, image_name)) for image_name in images_list]

        inputs = processor(text=[prompt], 
                        images=all_images, return_tensors="pt", padding=True)
    
        # Model prediction
        outputs = model(**inputs)

        # Image-text similarity score
        logits_per_image = outputs.logits_per_image

        # Take the softmax to get the label probabilities
        probs = logits_per_image.softmax(dim=1)

        # Filter the scores and return indices
        values, indices = logits_per_image.squeeze().topk(10)

        top_images, top_scores = [], []
        for score, index in zip(values, indices):
            top_images.append(images_list[int(index.numpy())])
            score = score.numpy().tolist()
            top_scores.append(round(score, 3))

        predictions.append([(image, score) for image, score in zip(top_images, top_scores)])

100%|██████████| 463/463 [06:09<00:00,  1.25it/s]


In [39]:
for pred in predictions[:10]:
    print(pred)

[('image.2166.jpg', 30.073), ('image.4417.jpg', 23.324), ('image.4415.jpg', 22.814), ('image.4413.jpg', 22.526), ('image.4414.jpg', 22.496), ('image.4418.jpg', 21.707), ('image.1150.jpg', 20.921), ('image.4419.jpg', 20.517), ('image.4416.jpg', 19.79), ('image.4412.jpg', 17.379)]
[('image.4420.jpg', 30.64), ('image.4429.png', 30.581), ('image.4423.jpg', 27.307), ('image.4427.jpg', 25.468), ('image.4424.jpg', 21.728), ('image.4422.jpg', 21.315), ('image.4426.jpg', 20.691), ('image.4425.jpg', 19.997), ('image.4421.jpg', 18.283), ('image.4428.jpg', 17.721)]
[('image.4432.jpg', 24.159), ('image.4438.jpg', 23.431), ('image.4431.jpg', 21.651), ('image.4434.jpg', 21.015), ('image.4437.jpg', 20.835), ('image.4433.jpg', 20.452), ('image.4435.jpg', 20.379), ('image.4430.jpg', 19.616), ('image.4436.jpg', 18.831), ('image.1166.jpg', 17.808)]
[('image.1435.jpg', 25.301), ('image.4446.png', 24.578), ('image.4445.jpg', 23.133), ('image.1434.jpg', 22.789), ('image.4441.jpg', 21.76), ('image.4442.jpg', 

In [40]:
from sklearn.metrics import accuracy_score

ground_truth = [gold for word, context, images_list, gold in test_data]
predicted = [pred[0][0] for pred in predictions]
accuracy = accuracy_score(ground_truth, predicted)
print(f'Model accuracy: {accuracy}')

Model accuracy: 0.5269978401727862


# TODO

1. Use wordnet to inject synonyms and make data augmentation
2. Train your own CLIP-like model